# Python Programming

In this course, you will learn basic programming skills in Python.

## Part 6: Advanced patterns

In this part we will learn some advanced patterns in Python, including

- classes
- coroutines
- iterators

### Classes

Classes can be seen as definitions of particular objects including constructors, attributes and methods. The general idea is that we define a class in order to create multiple instances of it, that all share some functionalities. Today the object-oriented programming (OOP) is used in most programs.

#### Class definition

The following cell defines the class `dog`. The class contains multiple methods (methods are functions of a class).

In [2]:
class Dog:                        # the definition of a class starts with the class keyword and the name of the class
    def __init__(self, name):     # the init method defines the creation of an instance of this class
        self.name = name      
        self.age = 0              # our dogs will have two attributes: name and age. 'self' refers to the new dog-object
        
    def celebrate_birthday(self): # this method will increase the age of the corresponding object by one
        self.age += 1
    
    def rename(self, new_name):   # this method takes a new name as argument and assignes it to the dogs name
        self.name = new_name
    
    def __str__(self):            # definition how to cast a dog to a printable string
        return '{}, {} years old'.format(self.name, self.age)

#### Class usage

Now we will create two instances of that class and use some of their methods.

In [3]:
dog1 = Dog('Bonny')               # initialization of a dog. Note that only one argument is given (the name)
dog2 = Dog('Clyde')               # 'self' is always passed implicitly

dog1.rename('Bonnie')             # call of dog1s rename method with dot operator 

for i in range(5):
    dog1.celebrate_birthday()
    dog2.celebrate_birthday()     # call celebrate_birthday() 5 times for both dogs

print(dog1)
print(dog2)                       # print will implicitly use our implemented string conversion

print(dog1.age)                   # access attribute of dog1 with dot operator

Bonnie, 5 years old
Clyde, 5 years old
5


Additional information about classes is available at the [official Python documentation](https://docs.python.org/3/tutorial/classes.html).

### Coroutines

Coroutines are a feature in Python that allows you to write asynchronous code in a more readable and manageable way. A couroutine can be suspended and resumed from the paused point for multiple times before the final termination. In this way, when multiple couroutines working together, they will be executed in an asynchronous manner and seemingly behave like concurrence. However, they are actually not, because Python's global interpreter lock (GIL) prevents multiple threads. Python's GIL will likely to be removed in the future, but this does not detract from the elegant nature of coroutines. Consider the following example

In [ ]:
import asyncio

# define a coroutine function
async def Count():  
    for i in range(5):
        print(i+1)
        # pause here for a while
        await asyncio.sleep(0.5)  
    return "Count done"

# define another coroutine function
async def Hello():  
    for i in range(5):
        print('Hello!')
        # pause here for a while
        await asyncio.sleep(0.5)
    return "Hello done"

# Run both coroutines on an event loop
# asyncio.run(asyncio.gather(Count(), Hello()))
# https://stackoverflow.com/questions/55409641/asyncio-run-cannot-be-called-from-a-running-event-loop-when-using-jupyter-no
await asyncio.gather(Count(), Hello())

There are two asynchronized function `Count()` and `Hello()` working together. From the output, they are seemingly executing simultaneously. When the program runs to `await asyncio.sleep(1)`, this function will be suspended, and other suspended functions will resume. That's why the outputs of these two functions are alternating. In modern Python (3.4+), we use the `async` keyword to define a coroutine (asynchronized function), and the `await` keyword for denoting the pause/resume point.

Coroutines usually work with *generators* behaving like producers in Python. In the context of coroutines, generators can be leveraged as asynchronous generators, combining the benefits of both concepts. Asynchronous generators use the `yield` keyword within an asynchronous function to produce a sequence of values asynchronously, making them particularly useful for scenarios like fetching data from external sources, handling asynchronous I/O operations, and managing concurrent tasks. Then another coroutine(s) will consume the generated data and process them. Together, generators and coroutines provide a versatile and elegant approach to working with both synchronous and asynchronous data streams in Python. Below is an example.

In [ ]:
import asyncio

async def async_data_generator():
    for i in range(5):
        # Simulate an asynchronous operation
        await asyncio.sleep(0.5)
        # Yield the current value asynchronously
        yield i

async def main():
    # Iterate over values produced by the asynchronous generator
    async for value in async_data_generator():
        print(f"Received value: {value}")

# Run the event loop to execute the asynchronous code
# asyncio.run(main())
await main()

### Iterators

An iterator is an object that can be iterated (looped) over in Python. It provides a way to access the elements of a iteratable object (e.g., `list`, `dict`). You can also define your own iterator applying your custom processing steps on data. For example,

In [4]:
class MyIterable:
    def __init__(self, year, height):
        self.index = 0
        self.year = year
        self.height = height
        assert len(self.year) == len(self.height)

    def __iter__(self):
        # make the class iterable
        return self

    def __next__(self):
        # output a combination of year and height correspondingly
        if self.index < len(self.year):
            thisYear = self.year[self.index]
            thisHeight = self.height[self.index]
            self.index += 1
            return thisYear, thisHeight
        else:
            raise StopIteration

    def __len__(self):
        return len(self.year)

# Create an iterable object
my_iterable = MyIterable([2000, 2001, 2002, 2003, 2004], [140, 145, 150, 155, 158])

# Iterate over the elements using a for loop
for element in my_iterable:
    print(element)
print("Length of iterable:", len(my_iterable))
# Access elements using indexing will fail, as MyIterable is not subscriptable.
# print("First element:", my_iterable[0])

(2000, 140)
(2001, 145)
(2002, 150)
(2003, 155)
(2004, 158)
Length of iterable: 5


You can modify `__next__()` inside your iterable class to modify the behavior for each iteration. This example here only simply iterates both lists together (zip-like). You can try to implement more advanced operations. Note that iterable objects are not always subscriptable. In the above example, `my_iterable[0]` will fail, as `__getitem__()` is not specified. Consider the following example:

In [ ]:
class MyIterable2:
    def __init__(self, year, height):
        self.index = 0
        self.year = year
        self.height = height
        assert len(self.year) == len(self.height)

    def __iter__(self):
        # make the class iterable
        return self

    def __next__(self):
        # output a combination of year and height correspondingly
        if self.index < len(self.year):
            thisYear = self.year[self.index]
            thisHeight = self.height[self.index]
            self.index += 1
            return thisYear, thisHeight
        else:
            raise StopIteration

    def __getitem__(self, index):
        # make the class subscriptable
        if index < len(self.year):
            return self.year[index], self.height[index]
        else:
            raise Exception("Out of range")
        

    def __len__(self):
        return len(self.year)

# Create an iterable object
my_iterable = MyIterable2([2000, 2001, 2002, 2003, 2004], [140, 145, 150, 155, 158])

print("First element:", my_iterable[0])

By implementing `__getitem__()`, the class now is subscriptable. You can also define a subscriptable class that cannot be iterated. Iterators are highly potent, straightforward to implement, and can be tailored for feeding data into deep learning models during stochastic gradient descent, which will be elaborated in the subsequent scripts.

You can also use `next()` to explicitly obtain the next item of the iterator.

In [ ]:
for _ in range(3):
    print(next(my_iterable))

### Tasks

#### Task 1: A simple class for bank accounts

Create a `BankAccount` class with attributes `account_number`, `account_holder`, and `balance`. Include methods for deposit, withdrawal, and displaying the account details.

In [ ]:
class BankAccount:
    pass
    # your code below (do not forget to delete pass above)

#### Task 2: Coroutine Chaining

Write two coroutines: one generates a series of random numbers and another squares the generated number. Chain these coroutines to produce the squared results. Output the corresponding squared result once a number is generated.

In [ ]:
# Hint: using yield. Think of async. generator above
# your code below

#### Task 3: Fibonacci Iterator

Write an iterator class `FibonacciNumbers` that generates Fibonacci numbers. Implement the `__iter__()` and `__next__()` methods.

In [ ]:
class FibonacciNumbers:
    pass
    # your code below (do not forget to delete pass above)